<a href="https://colab.research.google.com/github/antonpetkoff/identifier-suggestion/blob/master/notebooks/00-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# import utilities
import os
import shutil
from subprocess import check_output

In [2]:
!nvidia-smi

Sat May  9 20:20:37 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
%env WORKSPACE_DIR=/content/gdrive/My Drive/src

# TODO: how can one read an environment variable?!?
%cd '/content/gdrive/My Drive/src'

env: WORKSPACE_DIR=/content/gdrive/My Drive/src
/content/gdrive/My Drive/src


In [0]:
timestamp = check_output(['date', '-u', "+%Y-%m-%dT%H-%M-%S"]).decode('utf-8').strip()

os.environ['PROJECT_DIR'] = os.path.join(
    os.environ['WORKSPACE_DIR'],
    f'identifier-suggestion-{timestamp}',
)

In [6]:
!git clone https://github.com/antonpetkoff/identifier-suggestion.git --depth 1 "${PROJECT_DIR}"

Cloning into '/content/gdrive/My Drive/src/identifier-suggestion-2020-05-09T20-21-53'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 64 (delta 0), reused 39 (delta 0), pack-reused 0
Unpacking objects: 100% (64/64), done.


In [0]:
os.chdir(os.environ['PROJECT_DIR'])

In [8]:
!pwd
!ls -l

/content/gdrive/My Drive/src/identifier-suggestion-2020-05-09T20-21-53
total 54
-rw-------  1 root root 35149 May  9 20:22 LICENSE
drwx------  2 root root  4096 May  9 20:22 notebooks
-rw-------  1 root root  7506 May  9 20:22 README.md
drwx------  2 root root  4096 May  9 20:22 requirements
drwx------ 10 root root  4096 May  9 20:22 src


In [9]:
%tensorflow_version 2.x

import tensorflow
print(tensorflow.__version__)

2.2.0-rc4


In [10]:
# Google Colab has standard libraries like numpy, pandas, matplotlib and TF (of course) pre-installed
!pip install -r requirements/colab.txt

     |████████████████████████████████| 1.4MB 21.7MB/s 
     |████████████████████████████████| 4.3MB 60.1MB/s 
     |████████████████████████████████| 460kB 61.0MB/s 
     |████████████████████████████████| 112kB 42.5MB/s 
     |████████████████████████████████| 102kB 13.7MB/s 
     |████████████████████████████████| 102kB 12.2MB/s 
     |████████████████████████████████| 71kB 11.4MB/s 
     |████████████████████████████████| 71kB 10.5MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=12f42ce1d0c657c77795a1262302dadf43aaba4a67cdbe8e206a31fae50bacd3
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for watchdog: filename=watchdog-0.10.2-cp36-none-any.whl size=73605 sha256=031d348754fe121d8b681db65ca5a4a0666c0e596df14573f25e51ddb0dcd816
  Stored in directory: /root/.cache/pip/wheels/bc/ed/6c/028dea90d31b359cd2a7c8b0da4db80e41d24a59614154072e
  Created wheel for gq

In [11]:
# provide secrets to the project, e.g. access to wandb
shutil.copy(
    os.path.join(os.environ['WORKSPACE_DIR'], 'secrets/.env'),
    os.environ['PROJECT_DIR']
)

'/content/gdrive/My Drive/src/identifier-suggestion-2020-05-09T20-21-53/.env'

In [0]:
# monkey-patch (mock) os.symlink to be a noop, because wandb.save() uses it, but it is not supported by Google Colab Notebooks
os.symlink = lambda *x: print('Executing mocked noop symlink with arguments', x)

In [0]:
from argparse import Namespace
from src.pipelines.baseline import run

params = {
  'file_data_raw': '../data/interim/method-names-rich/elasticsearch.csv',
  'file_model_dir': 'models/saved/baseline/',
  'dir_preprocessed_data': '../data/interim/preprocessed/',
  'max_input_length': 200,
  'max_output_length': 8,
  'input_vocab_size': 20000,
  'input_embedding_dim': 128,
  'output_vocab_size': 6000,
  'output_embedding_dim': 128,
  'latent_dim': 512,
  'learning_rate': 0.0001,
  'epochs': 150,
  'batch_size': 512,
  'random_seed': 1,
}

run(Namespace(**params))

Experiment parameters:  Namespace(batch_size=512, dir_preprocessed_data='../data/interim/preprocessed/', epochs=150, file_data_raw='../data/interim/method-names-rich/elasticsearch.csv', file_model_dir='models/saved/baseline/', input_embedding_dim=128, input_vocab_size=20000, latent_dim=512, learning_rate=0.0001, max_input_length=200, max_output_length=8, output_embedding_dim=128, output_vocab_size=6000, random_seed=1)


Loading preprocessed files...
Loaded input vocabulary.
Loaded output vocabulary.
Loaded preprocessed files.
epoch 1 - batch 10 - loss 3.87074613571167 - precision 0.00010045053052017247 - recall 0.000499346078293133 - f1 0.00016721998029144776 - sparse_categorical_accuracy 0.1049107164144516
epoch 1 - batch 20 - loss 3.3388760089874268 - precision 0.00010517881956894015 - recall 0.000519560080534566 - f1 0.00017493221116596095 - sparse_categorical_accuracy 0.13918805122375488
epoch 1 - batch 30 - loss 3.254514455795288 - precision 0.00011164423550706118 - recall 0.0004927971776977217 - f1 0.0001808543995341652 - sparse_categorical_accuracy 0.16196057200431824
epoch 1 - batch 40 - loss 2.5491278171539307 - precision 0.00010480933337245733 - recall 0.00046788907826901565 - f1 0.00016992989052093228 - sparse_categorical_accuracy 0.16558314859867096
epoch 1 - batch 50 - loss 2.2286159992218018 - precision 0.0001414169696878701 - recall 0.0005295695900692761 - f1 0.00021803641559986812 - sp

requests_with_retry encountered retryable exception: 502 Server Error: Bad Gateway for url: https://api.wandb.ai/files/antonpetkoff/identifier-suggestion/2aybz60m/file_stream. args: ('https://api.wandb.ai/files/antonpetkoff/identifier-suggestion/2aybz60m/file_stream',), kwargs: {'json': {'files': {'output.log': {'offset': 677, 'content': ['2020-05-10T06:07:04.574469 epoch 34 - batch 140 - loss 0.2840855121612549 - precision 0.8344702813685186 - recall 0.8253799414150995 - f1 0.8183358409242707 - sparse_categorical_accuracy 0.48448461294174194\n']}, 'wandb-events.jsonl': {'offset': 946, 'content': ['{"system.gpu.0.gpu": 9.07, "system.gpu.0.memory": 5.33, "system.gpu.0.memoryAllocated": 58.38, "system.gpu.0.temp": 76.53, "system.gpu.0.powerWatts": 32.01, "system.gpu.0.powerPercent": 45.72, "system.cpu": 49.61, "system.memory": 23.7, "system.disk": 48.5, "system.proc.memory.availableMB": 9934.35, "system.proc.memory.rssMB": 4205.48, "system.proc.memory.percent": 32.3, "system.proc.cpu.thr

epoch 34 - batch 150 - loss 0.26505497097969055 - precision 0.835281597534221 - recall 0.8250178141370437 - f1 0.8187376793478373 - sparse_categorical_accuracy 0.48505207896232605


requests_with_retry encountered retryable exception: 502 Server Error: Bad Gateway for url: https://api.wandb.ai/files/antonpetkoff/identifier-suggestion/2aybz60m/file_stream. args: ('https://api.wandb.ai/files/antonpetkoff/identifier-suggestion/2aybz60m/file_stream',), kwargs: {'json': {'files': {'output.log': {'offset': 677, 'content': ['2020-05-10T06:07:04.574469 epoch 34 - batch 140 - loss 0.2840855121612549 - precision 0.8344702813685186 - recall 0.8253799414150995 - f1 0.8183358409242707 - sparse_categorical_accuracy 0.48448461294174194\n']}, 'wandb-events.jsonl': {'offset': 946, 'content': ['{"system.gpu.0.gpu": 9.07, "system.gpu.0.memory": 5.33, "system.gpu.0.memoryAllocated": 58.38, "system.gpu.0.temp": 76.53, "system.gpu.0.powerWatts": 32.01, "system.gpu.0.powerPercent": 45.72, "system.cpu": 49.61, "system.memory": 23.7, "system.disk": 48.5, "system.proc.memory.availableMB": 9934.35, "system.proc.memory.rssMB": 4205.48, "system.proc.memory.percent": 32.3, "system.proc.cpu.thr

epoch 34 - batch 160 - loss 0.1893574446439743 - precision 0.8371908662665543 - recall 0.826291294565228 - f1 0.820728618163007 - sparse_categorical_accuracy 0.4850917160511017
epoch 34 - batch 170 - loss 0.20359204709529877 - precision 0.8388824684512091 - recall 0.8270597919660176 - f1 0.8221906616837271 - sparse_categorical_accuracy 0.4825449585914612
epoch 34 - batch 180 - loss 0.14812086522579193 - precision 0.840752235115963 - recall 0.8284076719732487 - f1 0.8236478723368075 - sparse_categorical_accuracy 0.4827474057674408
epoch 34 - batch 190 - loss 0.15970642864704132 - precision 0.8440176473792342 - recall 0.8310033718842763 - f1 0.8266890510278276 - sparse_categorical_accuracy 0.48240867257118225
epoch 34 time: 1090.6579966545105 sec
epoch 35 - batch 10 - loss 0.39806100726127625 - precision 0.7304992888481813 - recall 0.7347877983187521 - f1 0.7160658805775545 - sparse_categorical_accuracy 0.5288783311843872
epoch 35 - batch 20 - loss 0.40743276476860046 - precision 0.75390